In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import spacy
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsSyntaxParser,
    NewsNERTagger,
    Doc,
    AddrExtractor
)
import json
import random
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import pymorphy2

from ner_end import df_small

morph = pymorphy2.MorphAnalyzer()

D:\Temp\NER\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('analyzer_analyzer_urls.csv')

In [46]:
df

http://0-9.ru  \
0         http://0009.ru   
1         http://001k.ru   
2        http://003ms.ru   
3        http://003rt.ru   
4          http://004.ru   
...                  ...   
9994    http://1hobby.ru   
9995      http://1hod.ru   
9996      http://1hop.ru   
9997  http://1hostels.ru   
9998   http://1hotel1.ru   

     <html><head>\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n        <title>Домен продается. Купить в магазине доменов RU-CENTER</title>\n        <meta name="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="keywords" content="домен, регистрация доменов, РФ, RU, COM, аукцион доменов, хостинг, почта, освобождающиеся домены">\n        <meta name="application-name" content="nic.ru">\n        <meta name="robots" content="noyaca">\n        <meta name="msapplication-square70x70logo" content="/favicon_70x70.png">\n        <meta name="msapplication-square150x150logo" content="/favicon_150x150.png">\n        <meta name="msapplication-wide310x150logo" content="/favicon_310x150.png">\n        <meta name="msapplication-square310x310logo" content="/favicon_310x310.png">\n        <meta property="og:title" id="title" content="Домен продается. Купить в магазине доменов RU-CENTER">\n        <meta property="og:description" id="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="twitter:card" content="summary_large_image">\n        <meta name="twitter:site" content="@ru_nic">\n        <meta name="twitter:title" content=" Домен продается. Купить в магазине доменов RU-CENTER ">\n        <meta name="twitter:description" content=" Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="twitter:image" content="https://storage.nic.ru/ru/images/jpg/zaglushkapark.jpg">\n        <meta property="og:image" content="https://storage.nic.ru/ru/images/jpg/zaglushkapark.jpg">\n        \n        <link rel="icon" type="image/png" href="https://www.nic.ru/favicon.png">\n        <link rel="icon" type="image/png" sizes="192x192" href="https://www.nic.ru/favicon_192x192.png">\n        <link rel="apple-touch-icon" type="image/png" href="https://www.nic.ru/favicon_192x192.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="152x152" href="https://www.nic.ru/favicon_152x152.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="167x167" href="https://www.nic.ru/favicon_167x167.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="180x180" href="https://www.nic.ru/favicon_180x180.png">\n        <link rel="preload" href="/static/frontend-entry/bundle.7709dbece824ed81e2fa.bundle.js" as="script">\n        <link rel="preconnect" href="https://mc.yandex.ru">\n        <link rel="preconnect" href="https://www.google.ru">\n        <link rel="preconnect" href="https://www.google.com">\n        <link rel="preconnect" href="https://www.google-analytics.com">\n        <link rel="preconnect" href="https://www.googletagmanager.com">\n        <link rel="preconnect" href="https://stats.g.doubleclick.net">\n        <meta name="theme-color" content="#ffffff">\n        <!-- Google Analytics -->\n        <!-- <script>window.user_id = '8739050';</script>-->\n        <!-- <script>window.login = '2709010/NIC-D/adm';</script>-->\n        <!-- <script>-->\n        <!-- !function(e,a,n,i,o,s){e.GoogleAnalyticsObject=i,e.ga=e.ga||function(){(e.ga.q=e.ga.q||[]).push(arguments)},e.ga.l=+new Date,o=a.createElement(n),s=a.getElementsByTagName(n)[0],o.async=1,o.src="//www.google-analytics.com/analytics.js",s.parentNode.insertBefore(o,s)}(window,document,"script","ga"),ga("create","UA-54851853-1","auto"),ga(function(e){var a=e.get("clientId");document.cookie="_ga_cid="+a+"; path=/",ga("set","dimension3",a)}),window.user_id&&(ga("set","&uid",window.user_id),ga("set","dimension4",window.user_id)),ga("set","dimension2",window.login?"User":"Guest"),window.send_ab_split_ga_events&&(window.

In [5]:
df_new = df.copy()

In [6]:
df_small = df_new.iloc[:100]

In [9]:
import re

In [10]:
def clean_html_content(html_text):
    if pd.isna(html_text) or not str(html_text).strip():
        return ""

    text = str(html_text)

    text = re.sub(r'<style[^>]*>.*?</style>', '', text, flags=re.DOTALL)
    text = re.sub(r'<script[^>]*>.*?</script>', '', text, flags=re.DOTALL)


    try:
        soup = BeautifulSoup(text, 'html.parser')

        for element in soup(['header', 'footer', 'nav', 'iframe', 'noscript', 'form']):
            element.decompose()

        text = '\n'.join(element.get_text(separator=' ', strip=True)
                        for element in soup.find_all(True) if element.name not in ['style', 'script'])
    except:
        # очистка, если проблема в парсере
        text = re.sub(r'<[^>]+>', ' ', text)
        print('error')


    text = text.lower()
    text = text.replace('"', '')

    return text

In [11]:
df_small.iloc[:, -1] = df_small.iloc[:, -1].apply(clean_html_content)

In [12]:
df_small

http://0-9.ru  \
0                  http://0009.ru   
1                  http://001k.ru   
2                 http://003ms.ru   
3                 http://003rt.ru   
4                   http://004.ru   
..                            ...   
95                http://001ru.ru   
96       http://001spb.edusite.ru   
97  http://0027.kontur-partner.ru   
98             http://003-stom.ru   
99                 http://0055.ru   

   <html><head>\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n        <title>Домен продается. Купить в магазине доменов RU-CENTER</title>\n        <meta name="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="keywords" content="домен, регистрация доменов, РФ, RU, COM, аукцион доменов, хостинг, почта, освобождающиеся домены">\n        <meta name="application-name" content="nic.ru">\n        <meta name="robots" content="noyaca">\n        <meta name="msapplication-square70x70logo" content="/favicon_70x70.png">\n        <meta name="msapplication-square150x150logo" content="/favicon_150x150.png">\n        <meta name="msapplication-wide310x150logo" content="/favicon_310x150.png">\n        <meta name="msapplication-square310x310logo" content="/favicon_310x310.png">\n        <meta property="og:title" id="title" content="Домен продается. Купить в магазине доменов RU-CENTER">\n        <meta property="og:description" id="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="twitter:card" content="summary_large_image">\n        <meta name="twitter:site" content="@ru_nic">\n        <meta name="twitter:title" content=" Домен продается. Купить в магазине доменов RU-CENTER ">\n        <meta name="twitter:description" content=" Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="twitter:image" content="https://storage.nic.ru/ru/images/jpg/zaglushkapark.jpg">\n        <meta property="og:image" content="https://storage.nic.ru/ru/images/jpg/zaglushkapark.jpg">\n        \n        <link rel="icon" type="image/png" href="https://www.nic.ru/favicon.png">\n        <link rel="icon" type="image/png" sizes="192x192" href="https://www.nic.ru/favicon_192x192.png">\n        <link rel="apple-touch-icon" type="image/png" href="https://www.nic.ru/favicon_192x192.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="152x152" href="https://www.nic.ru/favicon_152x152.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="167x167" href="https://www.nic.ru/favicon_167x167.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="180x180" href="https://www.nic.ru/favicon_180x180.png">\n        <link rel="preload" href="/static/frontend-entry/bundle.7709dbece824ed81e2fa.bundle.js" as="script">\n        <link rel="preconnect" href="https://mc.yandex.ru">\n        <link rel="preconnect" href="https://www.google.ru">\n        <link rel="preconnect" href="https://www.google.com">\n        <link rel="preconnect" href="https://www.google-analytics.com">\n        <link rel="preconnect" href="https://www.googletagmanager.com">\n        <link rel="preconnect" href="https://stats.g.doubleclick.net">\n        <meta name="theme-color" content="#ffffff">\n        <!-- Google Analytics -->\n        <!-- <script>window.user_id = '8739050';</script>-->\n        <!-- <script>window.login = '2709010/NIC-D/adm';</script>-->\n        <!-- <script>-->\n        <!-- !function(e,a,n,i,o,s){e.GoogleAnalyticsObject=i,e.ga=e.ga||function(){(e.ga.q=e.ga.q||[]).push(arguments)},e.ga.l=+new Date,o=a.createElement(n),s=a.getElementsByTagName(n)[0],o.async=1,o.src="//www.google-analytics.com/analytics.js",s.parentNode.insertBefore(o,s)}(window,document,"script","ga"),ga("create","UA-54851853-1","auto"),ga(function(e){var a=e.get("clientId");document.cookie="_ga_cid="+a+"; path=/",ga("set","dimension3",a)}),window.user_id&&(ga("set","&uid",window.user_id),ga("set","dimension4",window.user

# Организации

In [91]:
def extract_organizations(text):
    organizations = []
    words = text.split()

    for word in words:
        parsed = morph.parse(word)[0]

        # Проверка по тегу:
        if any(tag in parsed.tag for tag in ['Orgn']):
            organizations.append((word, "organization"))

        # Доп:
        elif (parsed.tag.POS == 'NOUN' and
              parsed.tag.case == 'nomn' and
              any(x in word.lower() for x in ['ооо', 'зао', 'ао', 'инк', 'ltd', 'оао'])):
            organizations.append((word, "organization"))

    return list(set(organizations))

In [92]:
def create_orgs_df(df):
    data = []
    for idx, url, text in zip(df.index, df.iloc[:, 0], df.iloc[:, -1]):

        if pd.notna(text):
            for org, org_type in extract_organizations(str(text)):
                data.append((idx, url, org, org_type))

    return pd.DataFrame(data, columns=['index', 'url', 'word', 'type']).drop_duplicates()

df_orgs = create_orgs_df(df_small)
df_orgs.to_csv('found_organizationsNEW.csv', index=False)


# Местоположение

In [ ]:
df_smallest = df_small.iloc[:5]

In [ ]:
df_smallest

In [13]:
from natasha import (
    Segmenter,
    MorphVocab,
    AddrExtractor
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

addr_extractor = AddrExtractor(morph_vocab)

def extract_address(text):
    match = addr_extractor.find(text)
    try:
        addr = ', '.join(f"{part.type or ''}: {part.value}" for part in match.fact.parts)
    except:
        addr = ''

    return [(addr, "address")]


In [15]:
def create_address_df(df):
    data = []
    for idx, url, text in zip(df.index, df.iloc[:, 0], df.iloc[:, -1]):
        if pd.notna(text):
            for address, address_type in extract_address(str(text)):
                data.append((idx, url, address, address_type))

    return pd.DataFrame(data, columns=['index', 'url', 'word', 'type']).drop_duplicates()


df1 = df_small[:10]
df2 = df_small[10:20]
df3 = df_small[20:30]


In [16]:
df_address = create_address_df(df1)
df_address.to_csv('found_addressNEW.csv', index=False)

In [17]:
df_address2 = create_address_df(df2)
df_address2.to_csv('found_addressNEW2.csv', index=False)

In [18]:
df_address3 = create_address_df(df3)
df_address3.to_csv('found_addressNEW3.csv', index=False)

In [19]:
df4 = df_small[30:50]

In [20]:
df_address4 = create_address_df(df4)
df_address4.to_csv('found_addressNEW4.csv', index=False)

In [22]:
df5 = df_small[50:75]
df_address5 = create_address_df(df5)
df_address5.to_csv('found_addressNEW5.csv', index=False)

In [23]:
df6 = df_small[75:]
df_address6 = create_address_df(df6)
df_address6.to_csv('found_addressNEW6.csv', index=False)

In [24]:
import pandas as pd

files = ["found_addressNEW.csv","found_addressNEW2.csv", "found_addressNEW3.csv", "found_addressNEW4.csv", "found_addressNEW5.csv", "found_addressNEW6.csv"]

dfs = []

for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

result = pd.concat(dfs, ignore_index=True)

result.to_csv("found_address_result.csv", index=False)

# Имена #

In [94]:
nlp = spacy.load("ru_core_news_sm")
nlp.max_length = 5000000
def extract_persons(text):
    doc = nlp(str(text))
    return [ent.text for ent in doc.ents if ent.label_ == "PER"]


In [101]:
def create_persons_df(df):
    data = []
    for idx, url, text in zip(df.index, df.iloc[:, 0], df.iloc[:, -1]):
        for person in extract_persons(str(text)):
            data.append((idx, url, person, 'name'))

    df_persons = pd.DataFrame(data, columns=['index', 'url', 'word', 'type'])
    return df_persons.drop_duplicates()
df_persons = create_persons_df(df_small)
df_persons.to_csv('found_personsNEW.csv', index=False)

# Должность #

In [104]:
nlp = spacy.load("ru_core_news_sm")
nlp.max_length = 5000000
def extract_positions(text):
    try:
        doc = nlp(str(text))
        positions = []

        for token in doc:
            # Выброс
            if token.pos_ != "NOUN" or token.morph.get("Case") != ["Nom"]:
                continue

            # 1: Управление глаголом назначения
            if any(child.lemma_ in {"назначить", "избрать", "утвердить"}
                   for child in token.head.children):
                positions.append((token.text, "position"))
                continue

            #2: Аппозитивная конструкция с PER/ORG
            if any(child.dep_ == "appos" and child.ent_type_ in {"PER", "ORG"}
                   for child in token.children):
                positions.append((token.text, "position"))
                continue

            # 3: Составное сказуемое с должностью
            if token.dep_ in {"nsubj", "attr"} and token.head.pos_ == "VERB":
                if token.head.lemma_ in {"работать"}:
                    positions.append((token.text, "position"))


        return list(set(positions))
    except:
        print('error')

In [105]:
def create_positions_df(df):
    data = []
    for idx, url, text in zip(df.index, df.iloc[:, 0],df.iloc[:, -1]):
        for position in extract_positions(str(text)):
            data.append((idx, url, position, 'position'))
    df_positions = pd.DataFrame(data, columns=['index', 'url', 'word', 'type'])
    return df_positions.drop_duplicates()

df_positions = create_positions_df(df_small)
df_positions.to_csv('found_positionsNEW.csv', index=False)

# Ссылки

In [102]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

class URLExtractor:
    def __init__(self, model_path="best_url_ner_model"):
        # Загружаем модель и токенизатор из локальной
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForTokenClassification.from_pretrained(model_path)

        # Создаем pipeline
        self.ner_pipeline = pipeline(
            "token-classification",
            model=self.model,
            tokenizer=self.tokenizer,
            aggregation_strategy="simple"  #  смежные токены --  группировка
        )

    def extract_urls(self, text):
        """извлечение URL из текста с постобработкой"""
        entities = self.ner_pipeline(text)

        # Фильтруем только URL (LABEL_1) и очищаем результаты
        urls = []
        for entity in entities:
            if entity["entity_group"] == "LABEL_1":
                url = entity["word"]

                # лишние пробелы
                url = url.replace(" ", "")

                # валидность (доп)
                if "://" in url or url.startswith(("http", "www", "ftp")):
                    urls.append(url)

        return urls

try:
    extractor = URLExtractor()
    print("Модель успешно загружена!")
except Exception as e:
    print(f"Ошибка загрузки модели: {e}")



Device set to use cpu


Модель успешно загружена!


In [103]:
def create_url_df(df):
    data = []

    for idx, prev_url, text in zip(df.index,df.iloc[:,0], df.iloc[:, -1]):
        extracted_urls = extractor.extract_urls(text)

        if not isinstance(extracted_urls, (list, tuple)):
            print(f"1Ошибка:  Значение: {extracted_urls}1")
            continue

        for item in extracted_urls:
            # print(f"item = {item}")
            url = item
            data.append((idx,prev_url ,url, 'url'))

    df_url = pd.DataFrame(data, columns=['index', 'url','word', 'type'])
    return df_url



df_url = create_url_df(df_small)
df_url.to_csv('found_urlsNEW.csv', index=False)

# Соединить в файлы

In [25]:
file_list = ["found_organizationsNEW.csv", "found_personsNEW.csv", "found_address_result.csv", "found_positionsNEW.csv"] # без ссылок

dataframes = []
for file in file_list:
    df = pd.read_csv(file)
    dataframes.append(df)

# Объединяем
full_df = pd.concat(dataframes, ignore_index=True)


result_df = full_df.pivot_table(index=["index", "url"],columns="type",values="word",aggfunc=lambda x: ", ".join(map(str, x))).reset_index()


result_df.columns = [col if isinstance(col, str) else col[1] for col in result_df.columns]


columns = ["name", "address", "organization", "position"]
for col in columns:
    if col not in result_df:
        result_df[col] = "-"
    else:
        result_df[col] = result_df[col].fillna("-")


result_df.to_csv("merged_result1.csv", index=False)


In [41]:

merged_df = pd.read_csv('merged_resultNEW.csv')
found_urls_df = pd.read_csv('found_urlsNEW.csv')

print(merged_df.columns)


print(found_urls_df.columns)


Index(['index', 'url', 'address', 'name', 'organization', 'position'], dtype='object')
Index(['index', 'url', 'word', 'type'], dtype='object')


In [26]:

merged_df = pd.read_csv('merged_result1.csv')
found_urls_df = pd.read_csv('found_urlsNEW.csv')


print("Столбцы merged_resultNEW.csv:", merged_df.columns)
print("Столбцы found_urlsNEW.csv:", found_urls_df.columns)

url_to_ssylka = found_urls_df.set_index('url')['word'].to_dict()

merged_df['ssylka'] = merged_df['url'].map(url_to_ssylka)

merged_df.to_csv('merged_result_all.csv', index=False)



Столбцы merged_resultNEW.csv: Index(['index', 'url', 'address', 'name', 'organization', 'position'], dtype='object')
Столбцы found_urlsNEW.csv: Index(['index', 'url', 'word', 'type'], dtype='object')


In [28]:
merged_df = pd.read_csv('merged_result_all.csv')

merged_df['ssylka'] = merged_df['ssylka'].fillna('-')

merged_df.to_csv('merged_result_with_ssylka.csv', index=False)

print(merged_df.head())


   index              url                                            address  \
0      0   http://0009.ru                                                NaN   
1      1   http://001k.ru                                                NaN   
2      2  http://003ms.ru  : москвы, город: москва, : москва, город: моск...   
3      3  http://003rt.ru  : москве, страна: рф, : москве, страна: рф, : ...   
4      4    http://004.ru  : краснодар, индекс: 103568, индекс: 101962, и...   

                                                name        organization  \
0                                                  -                   -   
1            каппучиино\n\nгладкий кирпич каппучиино                   -   
2                                                  -                   -   
3                                                  -  гуп, таттехмедфарм   
4  автоакустика автоакустика, алкотестер кабель с...                 свч   

                                            position ssylka  
